# Preparation of data for T5-based models fine-tuning

## Instalando pacotes

In [ ]:
!pip install --quiet transformers datasets sentencepiece

## Importando pacotes

In [ ]:
import pandas as pd
import json
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from datasets.dataset_dict import DatasetDict
from datasets import load_from_disk
from datasets import Dataset

from transformers import AutoTokenizer, MT5Tokenizer
from ast import literal_eval


## Carregando Dados

In [ ]:
root_folder = './'
data_folder = f"{root_folder}/datasets/"

# PTT5
model_name     = "unicamp-dl/ptt5-base-portuguese-vocab"
model_nickname = "ptt5"

# mT5
#model_name     = "google/mt5-small"
#model_nickname = "mt5"

model_folder = f"{root_folder}/{model_nickname}"

print(model_name)
print(model_folder)

In [ ]:
# carregando dados anotados e convertendo a coluna do ner para lista
train = pd.read_csv(f"{data_folder}train.csv", sep=';', keep_default_na=False, converters={"ner": literal_eval})
val   = pd.read_csv(f"{data_folder}val.csv", sep=';', keep_default_na=False, converters={"ner": literal_eval})
test  = pd.read_csv(f"{data_folder}test.csv", sep=';', keep_default_na=False, converters={"ner": literal_eval})

print(f"Train:{len(train)}, Val:{len(val)}, Test:{len(test)}")
print(len(train)+len(val)+len(test))

In [ ]:
train

## Preparando dados para treinamento/teste do Modelo

In [ ]:
import re
def get_dados(dados: pd.DataFrame):
    ds = {}
    ds['text'] = []
    ds['target'] = []
    for idx in range(0, len(dados)):

        text     = dados.loc[idx, 'text']
        new_text = text

        ner  = dados.loc[idx, 'ner']

        ent_label_len = 0
        for item in ner:
            start = item[0]  + ent_label_len
            end   = item[1]  + ent_label_len
            ent   = re.sub(r'\s+','_', item[2]) # entidade
            label = item[3] # label

            ent_label = f"[{ent}|{label}]"

            new_text = new_text[ : start ] + ent_label + new_text[ end : ]

            ent_label_len = ent_label_len + len(ent_label) - len(ent)

        ds['text'].append(text)
        ds['target'].append(new_text)


    return ds

#df_test = pd.DataFrame()

#df_test['text'] = ["O lucro líquido do Santander aumentou"]
#df_test['ner'] = "[[2, 15, 'lucro líquido', 'LUCRO'], [19, 28, 'Santander', 'COMPANY']]"

#df_test['text'] = ["Nós já solicitamos, temos autorização do Banco Central para fazer um novo processo de recompra, só que o bond do Banco está sendo negociado em torno de 104% do seu valor de fato."]
#df_test['ner'] = "[[41, 54, 'Banco Central', 'ORG'], [113, 118, 'Banco', 'COMPANY'], [152, 156, '104%', 'PERCENTUAL']]"
#df_test['ner'] = df_test['ner'].apply(literal_eval)
#result = get_dados(df_test)
#for key in result.keys():
#  print(result[key])

In [ ]:
dados_train = get_dados(train)
dados_val   = get_dados(val)
dados_test  = get_dados(test)

In [ ]:
if model_nickname == 'mt5':
  tokenizer = MT5Tokenizer.from_pretrained(model_name, legacy=False)
else:
  tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)

#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples, batch_size = 30000):
    all_tokenized_samples = {}
    all_tokenized_samples["text"]  = []
    all_tokenized_samples["target"] = []
    all_tokenized_samples["labels"] = []

    print(f"all samples len: {len(all_samples['text'])}")
    for i in range(0, len(all_samples["text"]), batch_size):

        samples_text   = all_samples['text'][i : i + batch_size]
        samples_target = all_samples['target'][i : i + batch_size]

        tokenized_text = tokenizer.batch_encode_plus(
                samples_text
                , max_length=512
                , truncation=True
            )

        tokenized_labels = tokenizer.batch_encode_plus(
                samples_target
                , max_length=512
                , truncation=True
            )

        labels = tokenized_labels['input_ids']

        for key in tokenized_text.keys():
            if key not in all_tokenized_samples.keys():
                all_tokenized_samples[key] = []

            all_tokenized_samples[key].extend(tokenized_text[key])
            

        all_tokenized_samples["text"].extend(samples_text)
        all_tokenized_samples["target"].extend(samples_target)
        all_tokenized_samples["labels"].extend(labels)

    return all_tokenized_samples


print("Tokenizando conjunto de treino...")
tokenized_train = tokenize_adjust_labels(dados_train)

print("Tokenizando conjunto de validação...")
tokenized_val   = tokenize_adjust_labels(dados_val)

print("Tokenizando conjunto de teste...")
tokenized_test  = tokenize_adjust_labels(dados_test)


## Salvando dados processados

In [ ]:
d = {'train':Dataset.from_dict({'text'          : tokenized_train['text'],
                                'target'        : tokenized_train['target'],
                                'input_ids'     : tokenized_train['input_ids'],
                                'attention_mask': tokenized_train['attention_mask'],
                                'labels'        : tokenized_train['labels']}),
     'validation':Dataset.from_dict({'text'          : tokenized_val['text'],
                                     'target'        : tokenized_val['target'],
                                     'input_ids'     : tokenized_val['input_ids'],
                                     'attention_mask': tokenized_val['attention_mask'],
                                     'labels'        : tokenized_val['labels']}),
     'test':Dataset.from_dict({'text'          : tokenized_test['text'],
                               'target'        : tokenized_test['target'],
                               'input_ids'     : tokenized_test['input_ids'],
                               'attention_mask': tokenized_test['attention_mask'],
                               'labels'        : tokenized_test['labels']})
     }

dataset = DatasetDict(d)
dataset

In [ ]:
# salvando dados processados em arquivos
dataset.save_to_disk(f"{model_folder}/dataset-bancos-{model_nickname}")

## Analisando dados

In [ ]:
dataset = load_from_disk(f"{model_folder}/dataset-bancos-{model_nickname}")
dataset

In [ ]:
dataset['train']['target'][0]

In [ ]:
# calculando numero de tokens gerados com o NLTK, numero de subwords gerados pelo tokenizer e a proporção entre eles
rows = []
for conjunto_nome in dataset.keys():
    conjunto_tokens     = dataset[conjunto_nome]['text']
    conjunto_input_ids  = dataset[conjunto_nome]['input_ids']
    conjunto_target     = dataset[conjunto_nome]['target']
    conjunto_labels     = dataset[conjunto_nome]['labels']
    len_conjunto        = len(conjunto_tokens)

    for i in range(len_conjunto):
        print(f"\r{conjunto_nome}: {i}/{len_conjunto}", end="")

        num_tokens_text   = len(word_tokenize(conjunto_tokens[i]))
        num_subwords_text = len(conjunto_input_ids[i]) - 1 # menos um token especial </s>
        proporcao_input   = num_subwords_text / num_tokens_text

        num_tokens_target   = len(word_tokenize(conjunto_target[i]))
        num_subwords_target = len(conjunto_labels[i]) - 1 # menos um token especial </s>
        proporcao_target    = num_subwords_target / num_tokens_target

        row = {'conjunto':              conjunto_nome
               , "num_tokens_input":    num_tokens_text
               , "num_subwords_input":  num_subwords_text
               , "proporcao_input":     proporcao_input
               , "num_tokens_target":   num_tokens_target
               , "num_subwords_target": num_subwords_target
               , "proporcao_target":    proporcao_target
               }

        rows.append(row)
    print()

df_metrics = pd.DataFrame.from_dict(rows)
df_metrics

In [ ]:
df_metrics.to_csv(f"{model_folder}/proporcao-tokens-{model_nickname}.csv", sep=';', index=False)

In [ ]:
print(f"Input: Tokens: {df_metrics['num_tokens_input'].sum()}   Subwords: {df_metrics['num_subwords_input'].sum()}")
print(f"Proporção input: {df_metrics['num_subwords_input'].sum() / df_metrics['num_tokens_input'].sum()}")

print(f"Target: Tokens: {df_metrics['num_tokens_target'].sum()}   Subwords: {df_metrics['num_subwords_target'].sum()}")
print(f"Proporção target: {df_metrics['num_subwords_target'].sum() / df_metrics['num_tokens_target'].sum()}")

In [ ]:
df_metrics.describe()

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained(model_name, max_length=512, legacy=False)

len(tokenizer.get_vocab())